In [1]:
import base.ciboulettepy as Cbl

In [2]:
cbl = Cbl.ciboulette()

In [3]:
table_cbl = cbl.ciboulettetable()

In [4]:
cbl.server = '192.168.1.134:11111'

In [5]:
cbl.server

'192.168.1.134:11111'

In [7]:
table_cbl

DRV_NAME,SERVER,DEVICE,FOCAL,SITE_LAT,SITE_LONG,INSTRUME,NAXIS1,NAXIS2,BINXY,PIXELXY,FILTER,NAME,OBSERVER,DATASET,ARCHIVES,RA,DEC,OBECT
str16,str18,int64,float64,float64,float64,str10,int64,int64,int64,float64,str3,str12,str4,str13,str22,float64,float64,str4
FITS astropy 4.2,192.168.1.18:11111,0,100.0,49.5961,-0.3531,Atik 383L+,3326,2504,1,5.4,CLS,CIBOULETTE-A,CAM1,/home/dataset,/home/dataset/archives,0.0,90.0,INIT
